In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import numpy as np

# get prophet model
from prophet import Prophet
from snowflake.ml.registry import Registry
from snowflake.ml.model import custom_model
from snowflake.ml.model import model_signature

# Viz
import seaborn as sns
import matplotlib.pyplot as plt

# Active session
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
from datetime import date
today = date.today()
print(today)

db ='DEMO_ENERGY'
schema= 'GOLD'
warehouse = 'DEMO_COMPUTE_XSMALL'

## function to create data
def create_datedf(max_date, period=7): 
    # Create a new DataFrame starting from max_date + 1 day, adding 7 days to each subsequent row
    date_range = pd.date_range(start=max_date + pd.Timedelta(days=1), periods=period)
    new_df = pd.DataFrame({'ds': date_range})
    return new_df


forecast_dates = create_datedf(today)
forecast_dates.head() 

In [ ]:
reg = Registry(session,database_name = db,schema_name= schema)
model_name='PROPHET_FORCAST_MODEL'
version = 'V1'
mv = reg.get_model('PROPHET_FORCAST_MODEL').version('V1')

predicted_volume = mv.run(forecast_dates)

In [ ]:
predicted_volume['Created_date']=today
predicted_volume.head()

In [ ]:
sdf = session.createDataFrame(predicted_volume)
sdf.write.mode("append").save_as_table("my_forecast_volume", table_type="transient")

In [ ]:
select * 
FROM demo_energy.gold.my_forecast_volume

In [ ]:
forecast = predicted_volume
# Plot the forecasted data (predicted values)
plt.plot(forecast['ds'], forecast['yhat'], color='orange', label='Forecasted Sales')  # Red line for forecast

# Optionally, add confidence intervals (forecast uncertainty)
plt.fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'], color='gray', alpha=0.3, label='Confidence Interval')

# Add labels and title
plt.xlabel('Date')
plt.ylabel('Sales')
plt.title('Oil Forecast vs Original Oil Volume')
plt.legend()

# Show the plot
plt.xticks(rotation=45)
plt.show()

In [ ]:
ls @DEMO_ENERGY.GOLD.MODEL_STAGE;